In [1]:
# -*- coding: utf-8
# Reinaldo Chaves (@paidatocandeira)
# Programa que procura possíveis indícios de:
# doações exageradas de Fundos públicos + poucos votos + gastos estranhos com fornecedores de campanha
#
#
# In Brazil, the term "laranja" designates the alias, that is, a person/company used to launder money

### Aqui são mostrados apenas indícios, é preciso mais investigação posterior para qualquer conclusão final
### Here only clues are shown, further investigation is needed for any final conclusion

In [2]:
import pandas as pd
from pathlib import Path

Formato de apresentação de valores com quatro decimais

In [3]:
pd.options.display.float_format = '{:,.4f}'.format

Dados coletados do Repositório de dados eleitorais: <br> 
http://www.tse.jus.br/eleicoes/estatisticas/repositorio-de-dados-eleitorais-1/repositorio-de-dados-eleitorais

## Acessa todos os candidatos de 2018

In [4]:
candidatos = pd.read_csv("consulta_cand_2018_BRASIL.csv",\
                         sep=';',\
                         encoding = 'latin_1',\
                         converters={'NR_CPF_CANDIDATO': lambda x: str(x), 
                                     'SQ_CANDIDATO': lambda x: str(x),
                                     'NR_TITULO_ELEITORAL_CANDIDATO': lambda x: str(x),
                                     'NR_CANDIDATO': lambda x: str(x)},\
                         usecols = ['SG_UF', 
                                    'DS_CARGO', 
                                    'SQ_CANDIDATO', 
                                    'NM_CANDIDATO', 
                                    'NM_URNA_CANDIDATO', 
                                    'NR_CPF_CANDIDATO', 
                                    'SG_PARTIDO', 
                                    'DT_NASCIMENTO', 
                                    'NR_IDADE_DATA_POSSE', 
                                    'NR_TITULO_ELEITORAL_CANDIDATO', 
                                    'DS_GENERO', 
                                    'DS_SIT_TOT_TURNO',
                                    'DS_SITUACAO_CANDIDATURA',
                                    'DS_DETALHE_SITUACAO_CAND'])

In [5]:
candidatos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29145 entries, 0 to 29144
Data columns (total 14 columns):
SG_UF                            29145 non-null object
DS_CARGO                         29145 non-null object
SQ_CANDIDATO                     29145 non-null object
NM_CANDIDATO                     29145 non-null object
NM_URNA_CANDIDATO                29145 non-null object
NR_CPF_CANDIDATO                 29145 non-null object
DS_SITUACAO_CANDIDATURA          29145 non-null object
DS_DETALHE_SITUACAO_CAND         29145 non-null object
SG_PARTIDO                       29145 non-null object
DT_NASCIMENTO                    29145 non-null object
NR_IDADE_DATA_POSSE              29145 non-null int64
NR_TITULO_ELEITORAL_CANDIDATO    29145 non-null object
DS_GENERO                        29145 non-null object
DS_SIT_TOT_TURNO                 29145 non-null object
dtypes: int64(1), object(13)
memory usage: 3.1+ MB


Separa apenas deputados federais e eleitos em 2018

In [6]:
candidatos_eleitos = candidatos[(candidatos['DS_CARGO'] == 'DEPUTADO FEDERAL') & \
                                ((candidatos['DS_SIT_TOT_TURNO'] == 'ELEITO POR MÉDIA') | \
                                 (candidatos['DS_SIT_TOT_TURNO'] == 'ELEITO POR QP')) ]

In [7]:
candidatos_eleitos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 513 entries, 11 to 29134
Data columns (total 14 columns):
SG_UF                            513 non-null object
DS_CARGO                         513 non-null object
SQ_CANDIDATO                     513 non-null object
NM_CANDIDATO                     513 non-null object
NM_URNA_CANDIDATO                513 non-null object
NR_CPF_CANDIDATO                 513 non-null object
DS_SITUACAO_CANDIDATURA          513 non-null object
DS_DETALHE_SITUACAO_CAND         513 non-null object
SG_PARTIDO                       513 non-null object
DT_NASCIMENTO                    513 non-null object
NR_IDADE_DATA_POSSE              513 non-null int64
NR_TITULO_ELEITORAL_CANDIDATO    513 non-null object
DS_GENERO                        513 non-null object
DS_SIT_TOT_TURNO                 513 non-null object
dtypes: int64(1), object(13)
memory usage: 60.1+ KB


## Doacoes dos candidatos em 2018

In [8]:
cand_doacoes = pd.read_csv("receitas_candidatos_2018_BRASIL.csv",\
                           sep=';', \
                           encoding = 'latin_1', \
                           converters={'SQ_CANDIDATO': lambda x: str(x), 
                                       'NR_CPF_CANDIDATO': lambda x: str(x), 
                                       'NR_CPF_CNPJ_DOADOR': lambda x: str(x),
                                       'SQ_CANDIDATO_DOADOR': lambda x: str(x),
                                       'SQ_RECEITA': lambda x: str(x)},\
                           decimal = ',',\
                           usecols = ['SG_UF', 
                                      'SQ_CANDIDATO', 
                                      'NM_CANDIDATO', 
                                      'NR_CPF_CANDIDATO', 
                                      'SG_PARTIDO', 
                                      'DS_FONTE_RECEITA', 
                                      'DS_ORIGEM_RECEITA', 
                                      'DS_ESPECIE_RECEITA', 
                                      'DS_CNAE_DOADOR', 
                                      'NR_CPF_CNPJ_DOADOR', 
                                      'NM_DOADOR_RFB', 
                                      'NM_PARTIDO_DOADOR', 
                                      'DS_RECEITA', 
                                      'VR_RECEITA',
                                      'NM_DOADOR'])

In [9]:
cand_doacoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316773 entries, 0 to 316772
Data columns (total 15 columns):
SG_UF                 316773 non-null object
SQ_CANDIDATO          316773 non-null object
NM_CANDIDATO          316773 non-null object
NR_CPF_CANDIDATO      316773 non-null object
SG_PARTIDO            316773 non-null object
DS_FONTE_RECEITA      316773 non-null object
DS_ORIGEM_RECEITA     316773 non-null object
DS_ESPECIE_RECEITA    316773 non-null object
DS_CNAE_DOADOR        316773 non-null object
NR_CPF_CNPJ_DOADOR    316773 non-null object
NM_DOADOR             316773 non-null object
NM_DOADOR_RFB         316773 non-null object
NM_PARTIDO_DOADOR     316773 non-null object
DS_RECEITA            316773 non-null object
VR_RECEITA            316773 non-null float64
dtypes: float64(1), object(14)
memory usage: 36.3+ MB


Conta tipos

In [10]:
cand_doacoes.groupby('DS_FONTE_RECEITA').size()

DS_FONTE_RECEITA
Fundo Especial       68608
Fundo Partidario     26126
Outros Recursos     222039
dtype: int64

In [11]:
cand_doacoes.groupby('DS_ORIGEM_RECEITA').size()

DS_ORIGEM_RECEITA
Comercialização de bens ou realização de eventos        97
Doações pela Internet                                  651
Recursos de Financiamento Coletivo                    5474
Recursos de origens não identificadas                  600
Recursos de outros candidatos                        51444
Recursos de partido político                         68567
Recursos de pessoas físicas                         159128
Recursos próprios                                    30430
Rendimentos de aplicações financeiras                  382
dtype: int64

## Doacoes vindas dos Fundos

Fundo Especial de Assistência Financeira aos Partidos Políticos e o Fundo Especial de Financiamento de Campanha

In [12]:
doacoes_fundos = cand_doacoes[(cand_doacoes.DS_FONTE_RECEITA == 'Fundo Especial')| \
                              (cand_doacoes.DS_FONTE_RECEITA == 'Fundo Partidario')]

doacoes_fundos = doacoes_fundos[(doacoes_fundos.NM_DOADOR.str.contains('Direção'))]

In [13]:
doacoes_fundos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59574 entries, 0 to 316764
Data columns (total 15 columns):
SG_UF                 59574 non-null object
SQ_CANDIDATO          59574 non-null object
NM_CANDIDATO          59574 non-null object
NR_CPF_CANDIDATO      59574 non-null object
SG_PARTIDO            59574 non-null object
DS_FONTE_RECEITA      59574 non-null object
DS_ORIGEM_RECEITA     59574 non-null object
DS_ESPECIE_RECEITA    59574 non-null object
DS_CNAE_DOADOR        59574 non-null object
NR_CPF_CNPJ_DOADOR    59574 non-null object
NM_DOADOR             59574 non-null object
NM_DOADOR_RFB         59574 non-null object
NM_PARTIDO_DOADOR     59574 non-null object
DS_RECEITA            59574 non-null object
VR_RECEITA            59574 non-null float64
dtypes: float64(1), object(14)
memory usage: 7.3+ MB


Soma o valor das doações de Fundos de cada candidato

In [14]:
soma = doacoes_fundos.groupby("SQ_CANDIDATO").VR_RECEITA.sum().reset_index()
soma.columns = ['SQ_CANDIDATO', 'VR_RECEITA_FUNDOS']

## Votos nas eleições de 2018

In [15]:
def _reader_2018(fname):
    return pd.read_csv(fname, \
                       sep=';', \
                       encoding='latin_1', \
                       converters={'SQ_CANDIDATO': lambda x: str(x),
                                   'NR_ZONA': lambda x: str(x)}, \
                       usecols = ['SG_UF', 
                                  'NM_MUNICIPIO', 
                                  'NR_ZONA', 
                                  'SQ_CANDIDATO', 
                                  'NM_CANDIDATO', 
                                  'NM_URNA_CANDIDATO', 
                                  'SG_PARTIDO', 
                                  'QT_VOTOS_NOMINAIS'])

Reúne todos os arquivos dos Estados em um único dataframe

In [16]:
folder = Path("votacao_candidato_munzona_2018")
votos = pd.concat([
    _reader_2018(csv)
    for csv in folder.glob("*.csv")
])

In [17]:
votos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9062279 entries, 0 to 21963
Data columns (total 8 columns):
SG_UF                object
NM_MUNICIPIO         object
NR_ZONA              object
SQ_CANDIDATO         object
NM_CANDIDATO         object
NM_URNA_CANDIDATO    object
SG_PARTIDO           object
QT_VOTOS_NOMINAIS    int64
dtypes: int64(1), object(7)
memory usage: 622.3+ MB


Soma os votos por candidato

In [18]:
conta = votos.groupby("SQ_CANDIDATO").QT_VOTOS_NOMINAIS.sum().reset_index()
conta.columns = ['SQ_CANDIDATO', 'QT_VOTOS_NOMINAIS']

In [19]:
conta.reset_index().head()

,index,SQ_CANDIDATO,QT_VOTOS_NOMINAIS
0,0,100000601175,6861
1,1,100000601176,88
2,2,100000601177,213
3,3,100000601178,28973
4,4,100000601179,2304


## Une eleitos e votos

In [20]:
votos_eleitos = pd.merge(candidatos_eleitos, conta, left_on='SQ_CANDIDATO', right_on='SQ_CANDIDATO')

In [21]:
votos_eleitos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 513 entries, 0 to 512
Data columns (total 15 columns):
SG_UF                            513 non-null object
DS_CARGO                         513 non-null object
SQ_CANDIDATO                     513 non-null object
NM_CANDIDATO                     513 non-null object
NM_URNA_CANDIDATO                513 non-null object
NR_CPF_CANDIDATO                 513 non-null object
DS_SITUACAO_CANDIDATURA          513 non-null object
DS_DETALHE_SITUACAO_CAND         513 non-null object
SG_PARTIDO                       513 non-null object
DT_NASCIMENTO                    513 non-null object
NR_IDADE_DATA_POSSE              513 non-null int64
NR_TITULO_ELEITORAL_CANDIDATO    513 non-null object
DS_GENERO                        513 non-null object
DS_SIT_TOT_TURNO                 513 non-null object
QT_VOTOS_NOMINAIS                513 non-null int64
dtypes: int64(2), object(13)
memory usage: 64.1+ KB


In [22]:
votos_eleitos.reset_index().head()

,index,SG_UF,DS_CARGO,SQ_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,NR_CPF_CANDIDATO,DS_SITUACAO_CANDIDATURA,DS_DETALHE_SITUACAO_CAND,SG_PARTIDO,DT_NASCIMENTO,NR_IDADE_DATA_POSSE,NR_TITULO_ELEITORAL_CANDIDATO,DS_GENERO,DS_SIT_TOT_TURNO,QT_VOTOS_NOMINAIS
0,0,MA,DEPUTADO FEDERAL,100000601912,CLEBER VERDE CORDEIRO MENDES,CLEBER VERDE,48357600344,APTO,DEFERIDO,PRB,10/05/1972,46,021413081180,MASCULINO,ELEITO POR QP,101806
1,1,RS,DEPUTADO FEDERAL,210000605296,OSMAR GASPARINI TERRA,OSMAR TERRA,19971478072,APTO,DEFERIDO,MDB,18/02/1950,68,021995280418,MASCULINO,ELEITO POR MÉDIA,86305
2,2,SP,DEPUTADO FEDERAL,250000613378,EDUARDO NANTES BOLSONARO,EDUARDO BOLSONARO,10655365770,APTO,DEFERIDO,PSL,10/07/1984,34,114611170337,MASCULINO,ELEITO POR QP,1843735
3,3,PA,DEPUTADO FEDERAL,140000606364,WAGNER BACH MARTINS,VAVÁ MARTINS,00019863098,APTO,DEFERIDO,PRB,03/06/1982,36,078779520493,MASCULINO,ELEITO POR QP,158717
4,4,BA,DEPUTADO FEDERAL,50000609224,ULDURICO ALENCAR PINTO,ULDÚRICO JÚNIOR,03746803101,APTO,DEFERIDO,PPL,30/01/1992,27,133647720523,MASCULINO,ELEITO POR QP,66343


## Une com doacoes de Fundos

In [23]:
votos_eleitos_doacoes_fundos = pd.merge(votos_eleitos, \
                                        soma, \
                                        left_on='SQ_CANDIDATO', \
                                        right_on='SQ_CANDIDATO')

In [24]:
votos_eleitos_doacoes_fundos.reset_index().head()

,index,SG_UF,DS_CARGO,SQ_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,NR_CPF_CANDIDATO,DS_SITUACAO_CANDIDATURA,DS_DETALHE_SITUACAO_CAND,SG_PARTIDO,DT_NASCIMENTO,NR_IDADE_DATA_POSSE,NR_TITULO_ELEITORAL_CANDIDATO,DS_GENERO,DS_SIT_TOT_TURNO,QT_VOTOS_NOMINAIS,VR_RECEITA_FUNDOS
0,0,MA,DEPUTADO FEDERAL,100000601912,CLEBER VERDE CORDEIRO MENDES,CLEBER VERDE,48357600344,APTO,DEFERIDO,PRB,10/05/1972,46,021413081180,MASCULINO,ELEITO POR QP,101806,"945,000.0000"
1,1,RS,DEPUTADO FEDERAL,210000605296,OSMAR GASPARINI TERRA,OSMAR TERRA,19971478072,APTO,DEFERIDO,MDB,18/02/1950,68,021995280418,MASCULINO,ELEITO POR MÉDIA,86305,"1,500,000.0000"
2,2,SP,DEPUTADO FEDERAL,250000613378,EDUARDO NANTES BOLSONARO,EDUARDO BOLSONARO,10655365770,APTO,DEFERIDO,PSL,10/07/1984,34,114611170337,MASCULINO,ELEITO POR QP,1843735,"7,950.4800"
3,3,PA,DEPUTADO FEDERAL,140000606364,WAGNER BACH MARTINS,VAVÁ MARTINS,00019863098,APTO,DEFERIDO,PRB,03/06/1982,36,078779520493,MASCULINO,ELEITO POR QP,158717,"789,031.0000"
4,4,BA,DEPUTADO FEDERAL,50000609224,ULDURICO ALENCAR PINTO,ULDÚRICO JÚNIOR,03746803101,APTO,DEFERIDO,PPL,30/01/1992,27,133647720523,MASCULINO,ELEITO POR QP,66343,"98,000.0000"


## Custo por voto

In [25]:
def custo(linha):
    votos = linha["QT_VOTOS_NOMINAIS"]
    valor = linha["VR_RECEITA_FUNDOS"]
    return valor / votos

votos_eleitos_doacoes_fundos["custo_por_voto"] = votos_eleitos_doacoes_fundos.apply(custo, axis=1)

Maiores custos por voto

In [26]:
votos_eleitos_doacoes_fundos.reset_index().sort_values("custo_por_voto", ascending=False).head(10)

,index,SG_UF,DS_CARGO,SQ_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,NR_CPF_CANDIDATO,DS_SITUACAO_CANDIDATURA,DS_DETALHE_SITUACAO_CAND,SG_PARTIDO,DT_NASCIMENTO,NR_IDADE_DATA_POSSE,NR_TITULO_ELEITORAL_CANDIDATO,DS_GENERO,DS_SIT_TOT_TURNO,QT_VOTOS_NOMINAIS,VR_RECEITA_FUNDOS,custo_por_voto
276,276,RR,DEPUTADO FEDERAL,230000604574,SHÉRIDAN ESTÉRFANY OLIVEIRA DE ANCHIETA,SHÉRIDAN,75096102220,APTO,DEFERIDO,PSDB,11/04/1984,34,003365412674,FEMININO,ELEITO POR QP,12129,"2,310,000.0000",190.4526
334,334,RR,DEPUTADO FEDERAL,230000608387,EDIO VIEIRA LOPES,EDIO LOPES,77548981872,APTO,DEFERIDO,PR,04/01/1952,67,000087082640,MASCULINO,ELEITO POR MÉDIA,11952,"1,800,000.0000",150.6024
361,361,RR,DEPUTADO FEDERAL,230000614391,HIRAN MANUEL GONÇALVES DA SILVA,DR. HIRAN GONÇALVES,14997169234,APTO,DEFERIDO,PP,19/06/1957,61,000168232682,MASCULINO,ELEITO POR MÉDIA,13299,"1,726,297.2000",129.8065
79,79,AP,DEPUTADO FEDERAL,30000617199,ANDRE DOS SANTOS ABDON,ANDRE ABDON,31601928220,APTO,DEFERIDO,PP,10/11/1970,48,003104632500,MASCULINO,ELEITO POR MÉDIA,12856,"1,634,541.0000",127.1423
353,353,AP,DEPUTADO FEDERAL,30000625407,ALINE PARANHOS VARONIL GURGEL,ALINE GURGEL,51150670215,APTO,DEFERIDO,PRB,25/11/1980,38,003029192500,FEMININO,ELEITO POR QP,16519,"1,952,976.2200",118.2261
219,219,AC,DEPUTADO FEDERAL,10000600848,MANUEL MARCOS CARVALHO DE MESQUITA,PASTOR MANUEL MARCOS,36089427268,APTO,DEFERIDO,PRB,07/05/1972,46,010751102267,MASCULINO,ELEITO POR MÉDIA,7489,"850,000.0000",113.4998
193,193,AP,DEPUTADO FEDERAL,30000602692,VINÍCIUS DE AZEVEDO GURGEL,VINÍCIUS,51004224249,APTO,DEFERIDO,PR,31/07/1978,40,003524192518,MASCULINO,ELEITO POR QP,18818,"1,650,000.0000",87.6820
202,202,AC,DEPUTADO FEDERAL,10000602515,FLAVIANO FLAVIO BAPTISTA DE MELO,FLAVIANO MELO,33251797700,APTO,DEFERIDO,MDB,17/11/1949,69,000745162429,MASCULINO,ELEITO POR MÉDIA,18723,"1,603,200.0000",85.6273
57,57,AC,DEPUTADO FEDERAL,10000602516,JESSICA ROJAS SALES,JÉSSICA SALES,65595564253,APTO,DEFERIDO,MDB,28/11/1980,38,003097672461,FEMININO,ELEITO POR QP,28717,"2,000,000.0000",69.6452
230,230,RR,DEPUTADO FEDERAL,230000606205,JOHNATHAN PEREIRA DE JESUS,JHONATAN DE JESUS,74274112268,APTO,DEFERIDO,PRB,03/09/1983,35,003063202682,MASCULINO,ELEITO POR QP,13429,"925,540.0000",68.9210


Maior custo então é de R$ 190,4

## Verifica em todos os candidatos a deputado federal do Brasil quem teve um custo por voto pelo menos dez vezes maior do que 190,4

In [27]:
candidatos_dep = candidatos[(candidatos['DS_CARGO'] == 'DEPUTADO FEDERAL') & 
                            (candidatos['DS_SITUACAO_CANDIDATURA'] == 'APTO')]

votos_todos = pd.merge(candidatos_dep, conta, left_on='SQ_CANDIDATO', right_on='SQ_CANDIDATO')

votos_todos_doacoes_fundos = pd.merge(votos_todos, \
                                      soma, \
                                      left_on='SQ_CANDIDATO', \
                                      right_on='SQ_CANDIDATO')

def custo(linha):
    votos = linha["QT_VOTOS_NOMINAIS"]
    valor = linha["VR_RECEITA_FUNDOS"]
    if votos != 0 and valor !=0:
        return valor / votos 
    else:
        return valor

votos_todos_doacoes_fundos["custo_por_voto"] = votos_todos_doacoes_fundos.apply(custo, axis=1)

votos_todos_doacoes_fundos.reset_index().sort_values("custo_por_voto", ascending=False).head(10)

,index,SG_UF,DS_CARGO,SQ_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,NR_CPF_CANDIDATO,DS_SITUACAO_CANDIDATURA,DS_DETALHE_SITUACAO_CAND,SG_PARTIDO,DT_NASCIMENTO,NR_IDADE_DATA_POSSE,NR_TITULO_ELEITORAL_CANDIDATO,DS_GENERO,DS_SIT_TOT_TURNO,QT_VOTOS_NOMINAIS,VR_RECEITA_FUNDOS,custo_por_voto
752,752,MS,DEPUTADO FEDERAL,120000613466,ALCIDES JESUS PERALTA BERNAL,ALCIDES BERNAL,34388800163,APTO,INDEFERIDO COM RECURSO,PP,14/07/1965,53,006372221988,MASCULINO,NÃO ELEITO,0,"1,312,040.0000","1,312,040.0000"
4189,4189,AP,DEPUTADO FEDERAL,30000613830,ANTONIO ROBERTO RODRIGUES GOES DA SILVA,ROBERTO,26404222272,APTO,INDEFERIDO COM RECURSO,PDT,22/06/1966,52,000205522542,MASCULINO,NÃO ELEITO,0,"500,000.0000","500,000.0000"
2371,2371,RJ,DEPUTADO FEDERAL,190000613606,WASHINGTON LUIZ CARDOSO SIQUEIRA,QUAQUA,03215292785,APTO,INDEFERIDO COM RECURSO,PT,31/05/1971,47,077246310345,MASCULINO,NÃO ELEITO,0,"471,000.0000","471,000.0000"
5014,5014,RJ,DEPUTADO FEDERAL,190000607877,NELSON ROBERTO BORNIER DE OLIVEIRA,BORNIER,10041800710,APTO,INDEFERIDO COM RECURSO,PROS,14/01/1950,69,052866010370,MASCULINO,NÃO ELEITO,0,"435,046.2500","435,046.2500"
4011,4011,SC,DEPUTADO FEDERAL,240000627221,JOÃO RODRIGUES,JOÃO RODRIGUES,23278951387,APTO,INDEFERIDO COM RECURSO,PSD,23/03/1967,51,024601780906,MASCULINO,NÃO ELEITO,0,"200,000.0000","200,000.0000"
4157,4157,SP,DEPUTADO FEDERAL,250000623474,JOSE CARLOS OROSCO,JUNIOR OROSCO,27098424838,APTO,INDEFERIDO COM RECURSO,PDT,02/03/1979,39,253970790124,MASCULINO,NÃO ELEITO,0,"200,000.0000","200,000.0000"
1636,1636,AP,DEPUTADO FEDERAL,30000617197,ANDERSON RICARDO DE ALMEIDA FEIO,ANDERSON ALMEIDA,66861829234,APTO,INDEFERIDO COM RECURSO,DEM,07/06/1980,38,003150712526,MASCULINO,NÃO ELEITO,0,"150,000.0000","150,000.0000"
470,470,SP,DEPUTADO FEDERAL,250000605436,EVALDO RUI VICENTINI,VICENTINI,34342184753,APTO,INDEFERIDO COM RECURSO,PSDB,28/11/1945,73,039067611007,MASCULINO,NÃO ELEITO,0,"100,000.0000","100,000.0000"
2467,2467,AM,DEPUTADO FEDERAL,40000618310,GERSON JOSE FEITOSA DE OLIVEIRA,GERSON FEITOSA,83547720206,APTO,INDEFERIDO COM RECURSO,MDB,06/02/1986,32,022728602208,MASCULINO,NÃO ELEITO,0,"100,000.0000","100,000.0000"
3757,3757,MG,DEPUTADO FEDERAL,130000606496,NATALICIO TENORIO CAVALCANTI FREITAS LIMA,TENORIO CAVALCANTI,83140646704,APTO,INDEFERIDO COM RECURSO,AVANTE,05/01/1965,54,081409970248,MASCULINO,NÃO ELEITO,0,"90,270.8300","90,270.8300"


Filtra por não eleitos e que tiveram custo por voto acima ou igual a R$ 1.904

In [28]:
nao_eleitos = votos_todos_doacoes_fundos[(votos_todos_doacoes_fundos['DS_SIT_TOT_TURNO'] == 'NÃO ELEITO')| 
                                         (votos_todos_doacoes_fundos['DS_SIT_TOT_TURNO'] == 'SUPLENTE')].copy()

In [29]:
nao_eleitos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4575 entries, 0 to 5033
Data columns (total 17 columns):
SG_UF                            4575 non-null object
DS_CARGO                         4575 non-null object
SQ_CANDIDATO                     4575 non-null object
NM_CANDIDATO                     4575 non-null object
NM_URNA_CANDIDATO                4575 non-null object
NR_CPF_CANDIDATO                 4575 non-null object
DS_SITUACAO_CANDIDATURA          4575 non-null object
DS_DETALHE_SITUACAO_CAND         4575 non-null object
SG_PARTIDO                       4575 non-null object
DT_NASCIMENTO                    4575 non-null object
NR_IDADE_DATA_POSSE              4575 non-null int64
NR_TITULO_ELEITORAL_CANDIDATO    4575 non-null object
DS_GENERO                        4575 non-null object
DS_SIT_TOT_TURNO                 4575 non-null object
QT_VOTOS_NOMINAIS                4575 non-null int64
VR_RECEITA_FUNDOS                4575 non-null float64
custo_por_voto                

In [30]:
seleciona = nao_eleitos['custo_por_voto'] >= 1904

valores_altos = nao_eleitos[seleciona]

In [31]:
valores_altos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34 entries, 159 to 5014
Data columns (total 17 columns):
SG_UF                            34 non-null object
DS_CARGO                         34 non-null object
SQ_CANDIDATO                     34 non-null object
NM_CANDIDATO                     34 non-null object
NM_URNA_CANDIDATO                34 non-null object
NR_CPF_CANDIDATO                 34 non-null object
DS_SITUACAO_CANDIDATURA          34 non-null object
DS_DETALHE_SITUACAO_CAND         34 non-null object
SG_PARTIDO                       34 non-null object
DT_NASCIMENTO                    34 non-null object
NR_IDADE_DATA_POSSE              34 non-null int64
NR_TITULO_ELEITORAL_CANDIDATO    34 non-null object
DS_GENERO                        34 non-null object
DS_SIT_TOT_TURNO                 34 non-null object
QT_VOTOS_NOMINAIS                34 non-null int64
VR_RECEITA_FUNDOS                34 non-null float64
custo_por_voto                   34 non-null float64
dtypes: f

In [33]:
valores_altos.reset_index().sort_values("custo_por_voto", ascending=False).head(10)

,index,SG_UF,DS_CARGO,SQ_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,NR_CPF_CANDIDATO,DS_SITUACAO_CANDIDATURA,DS_DETALHE_SITUACAO_CAND,SG_PARTIDO,DT_NASCIMENTO,NR_IDADE_DATA_POSSE,NR_TITULO_ELEITORAL_CANDIDATO,DS_GENERO,DS_SIT_TOT_TURNO,QT_VOTOS_NOMINAIS,VR_RECEITA_FUNDOS,custo_por_voto
6,752,MS,DEPUTADO FEDERAL,120000613466,ALCIDES JESUS PERALTA BERNAL,ALCIDES BERNAL,34388800163,APTO,INDEFERIDO COM RECURSO,PP,14/07/1965,53,006372221988,MASCULINO,NÃO ELEITO,0,"1,312,040.0000","1,312,040.0000"
27,4189,AP,DEPUTADO FEDERAL,30000613830,ANTONIO ROBERTO RODRIGUES GOES DA SILVA,ROBERTO,26404222272,APTO,INDEFERIDO COM RECURSO,PDT,22/06/1966,52,000205522542,MASCULINO,NÃO ELEITO,0,"500,000.0000","500,000.0000"
14,2371,RJ,DEPUTADO FEDERAL,190000613606,WASHINGTON LUIZ CARDOSO SIQUEIRA,QUAQUA,03215292785,APTO,INDEFERIDO COM RECURSO,PT,31/05/1971,47,077246310345,MASCULINO,NÃO ELEITO,0,"471,000.0000","471,000.0000"
33,5014,RJ,DEPUTADO FEDERAL,190000607877,NELSON ROBERTO BORNIER DE OLIVEIRA,BORNIER,10041800710,APTO,INDEFERIDO COM RECURSO,PROS,14/01/1950,69,052866010370,MASCULINO,NÃO ELEITO,0,"435,046.2500","435,046.2500"
26,4157,SP,DEPUTADO FEDERAL,250000623474,JOSE CARLOS OROSCO,JUNIOR OROSCO,27098424838,APTO,INDEFERIDO COM RECURSO,PDT,02/03/1979,39,253970790124,MASCULINO,NÃO ELEITO,0,"200,000.0000","200,000.0000"
23,4011,SC,DEPUTADO FEDERAL,240000627221,JOÃO RODRIGUES,JOÃO RODRIGUES,23278951387,APTO,INDEFERIDO COM RECURSO,PSD,23/03/1967,51,024601780906,MASCULINO,NÃO ELEITO,0,"200,000.0000","200,000.0000"
10,1636,AP,DEPUTADO FEDERAL,30000617197,ANDERSON RICARDO DE ALMEIDA FEIO,ANDERSON ALMEIDA,66861829234,APTO,INDEFERIDO COM RECURSO,DEM,07/06/1980,38,003150712526,MASCULINO,NÃO ELEITO,0,"150,000.0000","150,000.0000"
1,470,SP,DEPUTADO FEDERAL,250000605436,EVALDO RUI VICENTINI,VICENTINI,34342184753,APTO,INDEFERIDO COM RECURSO,PSDB,28/11/1945,73,039067611007,MASCULINO,NÃO ELEITO,0,"100,000.0000","100,000.0000"
16,2467,AM,DEPUTADO FEDERAL,40000618310,GERSON JOSE FEITOSA DE OLIVEIRA,GERSON FEITOSA,83547720206,APTO,INDEFERIDO COM RECURSO,MDB,06/02/1986,32,022728602208,MASCULINO,NÃO ELEITO,0,"100,000.0000","100,000.0000"
21,3757,MG,DEPUTADO FEDERAL,130000606496,NATALICIO TENORIO CAVALCANTI FREITAS LIMA,TENORIO CAVALCANTI,83140646704,APTO,INDEFERIDO COM RECURSO,AVANTE,05/01/1965,54,081409970248,MASCULINO,NÃO ELEITO,0,"90,270.8300","90,270.8300"


Aqui vemos que quase todos são candidatos que tiveram pendências na Justiça e por isso têm zero votos registrados

In [34]:
valores_altos.groupby('DS_DETALHE_SITUACAO_CAND').size()

DS_DETALHE_SITUACAO_CAND
DEFERIDO                   5
INDEFERIDO COM RECURSO    29
dtype: int64

In [35]:
valores_altos.groupby('QT_VOTOS_NOMINAIS').size()

QT_VOTOS_NOMINAIS
0      29
25      1
67      1
107     1
233     1
461     1
dtype: int64

No Repositório de dados eleitorais, em Candidatos-Motivo da cassação, é possível ver os motivos das pendências na Justiça

Seleciona apenas quem não teve zero votos

In [36]:
valores_altos = valores_altos[(valores_altos['QT_VOTOS_NOMINAIS'] != 0)]

In [38]:
valores_altos.reset_index().sort_values("custo_por_voto", ascending=False).head(10)

,index,SG_UF,DS_CARGO,SQ_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,NR_CPF_CANDIDATO,DS_SITUACAO_CANDIDATURA,DS_DETALHE_SITUACAO_CAND,SG_PARTIDO,DT_NASCIMENTO,NR_IDADE_DATA_POSSE,NR_TITULO_ELEITORAL_CANDIDATO,DS_GENERO,DS_SIT_TOT_TURNO,QT_VOTOS_NOMINAIS,VR_RECEITA_FUNDOS,custo_por_voto
4,4660,AP,DEPUTADO FEDERAL,30000617205,RICELY COSTA SANTOS,RICELY BORBOLETA,79948154215,APTO,DEFERIDO,PSD,03/09/1982,36,003585602534,FEMININO,SUPLENTE,107,"360,000.0000","3,364.4860"
3,4506,RJ,DEPUTADO FEDERAL,190000607880,ROSANGELA DA MOTTA REBELLO MACHADO,ROSANGELA MOTTA,51731762704,APTO,DEFERIDO,PROS,07/12/1958,60,057030890388,FEMININO,SUPLENTE,67,"175,781.2500","2,623.6007"
1,1452,RR,DEPUTADO FEDERAL,230000603361,D ANGELA ANALDINA DA SILVA KOTINSCKI,D ANGELA,38299453291,APTO,DEFERIDO,PSB,18/07/1976,42,002207882658,FEMININO,NÃO ELEITO,25,"60,584.0000","2,423.3600"
2,2119,RJ,DEPUTADO FEDERAL,190000626410,GERALDO RAMOS,GERALDO INDIO,90554582791,APTO,DEFERIDO,PRB,21/12/1966,52,055456240337,MASCULINO,SUPLENTE,461,"997,100.0000","2,162.9067"
0,159,AL,DEPUTADO FEDERAL,20000621940,MARIA APARECIDA LISBOA,TIDA DO BREJINHO,26816507800,APTO,DEFERIDO,PSD,17/01/1976,43,021890971783,FEMININO,SUPLENTE,233,"450,000.0000","1,931.3305"


## Acessa os gastos com fornecedores

In [39]:
despesas = pd.read_csv("despesas_contratadas_candidatos_2018_BRASIL.csv",\
                         sep=';',\
                         encoding = 'latin_1',\
                         converters={'NR_CNPJ_PRESTADOR_CONTA': lambda x: str(x), 
                                     'SQ_CANDIDATO': lambda x: str(x),
                                     'NR_CPF_CANDIDATO': lambda x: str(x),
                                     'NR_CPF_CNPJ_FORNECEDOR': lambda x: str(x),
                                     'NR_DOCUMENTO': lambda x: str(x)},\
                         usecols = ['SG_UF',
                                    'NR_CNPJ_PRESTADOR_CONTA',
                                    'DS_CARGO',
                                    'SQ_CANDIDATO',
                                    'NM_CANDIDATO',
                                    'NR_CPF_CANDIDATO',
                                    'SG_PARTIDO',
                                    'DS_TIPO_FORNECEDOR',
                                    'NR_CPF_CNPJ_FORNECEDOR',
                                    'NM_FORNECEDOR',
                                    'NM_FORNECEDOR_RFB',
                                    'SG_UF_FORNECEDOR',
                                    'NM_MUNICIPIO_FORNECEDOR',
                                    'NR_DOCUMENTO',
                                    'DS_ORIGEM_DESPESA',
                                    'DT_DESPESA',
                                    'DS_DESPESA',
                                    'VR_DESPESA_CONTRATADA'], \
                         decimal = ',')

In [40]:
despesas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1673557 entries, 0 to 1673556
Data columns (total 18 columns):
SG_UF                      1673557 non-null object
NR_CNPJ_PRESTADOR_CONTA    1673557 non-null object
DS_CARGO                   1673557 non-null object
SQ_CANDIDATO               1673557 non-null object
NM_CANDIDATO               1673557 non-null object
NR_CPF_CANDIDATO           1673557 non-null object
SG_PARTIDO                 1673557 non-null object
DS_TIPO_FORNECEDOR         1673557 non-null object
NR_CPF_CNPJ_FORNECEDOR     1673557 non-null object
NM_FORNECEDOR              1673557 non-null object
NM_FORNECEDOR_RFB          1673557 non-null object
SG_UF_FORNECEDOR           1673557 non-null object
NM_MUNICIPIO_FORNECEDOR    1673557 non-null object
NR_DOCUMENTO               1673557 non-null object
DS_ORIGEM_DESPESA          1673557 non-null object
DT_DESPESA                 1673557 non-null object
DS_DESPESA                 1673557 non-null object
VR_DESPESA_CONTRATAD

Seleciona as despesas de RICELY COSTA SANTOS - NÃO DECLAROU

In [43]:
seleciona = despesas[(despesas['SQ_CANDIDATO'] == '30000617205')]

In [44]:
seleciona

,SG_UF,NR_CNPJ_PRESTADOR_CONTA,DS_CARGO,SQ_CANDIDATO,NM_CANDIDATO,NR_CPF_CANDIDATO,SG_PARTIDO,DS_TIPO_FORNECEDOR,NR_CPF_CNPJ_FORNECEDOR,NM_FORNECEDOR,NM_FORNECEDOR_RFB,SG_UF_FORNECEDOR,NM_MUNICIPIO_FORNECEDOR,NR_DOCUMENTO,DS_ORIGEM_DESPESA,DT_DESPESA,DS_DESPESA,VR_DESPESA_CONTRATADA


Seleciona as despesas de ROSANGELA DA MOTTA REBELLO MACHADO

In [45]:
seleciona = despesas[(despesas['SQ_CANDIDATO'] == '190000607880')]
seleciona

,SG_UF,NR_CNPJ_PRESTADOR_CONTA,DS_CARGO,SQ_CANDIDATO,NM_CANDIDATO,NR_CPF_CANDIDATO,SG_PARTIDO,DS_TIPO_FORNECEDOR,NR_CPF_CNPJ_FORNECEDOR,NM_FORNECEDOR,NM_FORNECEDOR_RFB,SG_UF_FORNECEDOR,NM_MUNICIPIO_FORNECEDOR,NR_DOCUMENTO,DS_ORIGEM_DESPESA,DT_DESPESA,DS_DESPESA,VR_DESPESA_CONTRATADA
1075890,RJ,31194331000133,Deputado Federal,190000607880,ROSANGELA DA MOTTA REBELLO MACHADO,51731762704,PROS,Pessoa Jurídica,03746198000145,NASCIMENTO FELIX PROPAGANDA E PUBLICIDADE LTDA,NASCIMENTO FELIX PROPAGANDA E PUBLICIDADE LTDA,#NULO#,#NULO#,7 - SN,Criação e inclusão de páginas na internet,05/09/2018,CRIAÇÃO ID VISUAL + REDES SOCIAIS + WEBSITE,"75,000.0000"
1075891,RJ,31194331000133,Deputado Federal,190000607880,ROSANGELA DA MOTTA REBELLO MACHADO,51731762704,PROS,Pessoa Jurídica,-1,#NULO#,#NULO#,#NULO#,#NULO#,#NULO#,Encargos sociais,20/09/2018,DARF,"4,906.1500"
1075892,RJ,31194331000133,Deputado Federal,190000607880,ROSANGELA DA MOTTA REBELLO MACHADO,51731762704,PROS,Pessoa Jurídica,-1,#NULO#,#NULO#,#NULO#,#NULO#,#NULO#,"Encargos financeiros, taxas bancárias e/ou op....",04/09/2018,DOC/TED,17.8500
1075893,RJ,31194331000133,Deputado Federal,190000607880,ROSANGELA DA MOTTA REBELLO MACHADO,51731762704,PROS,Pessoa Jurídica,-1,#NULO#,#NULO#,#NULO#,#NULO#,#NULO#,"Encargos financeiros, taxas bancárias e/ou op....",04/09/2018,DOC/TED,17.8500
1075894,RJ,31194331000133,Deputado Federal,190000607880,ROSANGELA DA MOTTA REBELLO MACHADO,51731762704,PROS,Pessoa Física,01876825774,SEBASTIAO MARCOS CARVALHO FRANCISCO,SEBASTIAO MARCOS CARVALHO FRANCISCO,#NULO#,#NULO#,SN,Despesas com pessoal,27/08/2018,PANFLETAGEM DE 27/08 A 06/10,"1,900.0000"
1075895,RJ,31194331000133,Deputado Federal,190000607880,ROSANGELA DA MOTTA REBELLO MACHADO,51731762704,PROS,Pessoa Física,11205242708,CARINE SETTI TOFFANO BARBOSA,CARINE SETTI TOFFANO BARBOSA,#NULO#,#NULO#,SN,Despesas com pessoal,27/08/2018,PANFLETAGEM 17/08 A 06/10,"1,900.0000"
1075896,RJ,31194331000133,Deputado Federal,190000607880,ROSANGELA DA MOTTA REBELLO MACHADO,51731762704,PROS,Pessoa Física,66547350720,ROSIANA DE OLIVEIRA LEITE,ROSIANA DE OLIVEIRA LEITE,#NULO#,#NULO#,SN,Serviços prestados por terceiros,10/09/2018,ASSESSORIA JURIDICA,"15,369.3600"
1075897,RJ,31194331000133,Deputado Federal,190000607880,ROSANGELA DA MOTTA REBELLO MACHADO,51731762704,PROS,Pessoa Jurídica,03746198000145,NASCIMENTO FELIX PROPAGANDA E PUBLICIDADE LTDA,NASCIMENTO FELIX PROPAGANDA E PUBLICIDADE LTDA,#NULO#,#NULO#,5 - SN,Criação e inclusão de páginas na internet,03/09/2018,SERV CRIAÇÃO ID VISUAL + REDES SOCIAIS + WEBSITE,"75,000.0000"


## Detalha gastos para buscar possíveis casos estranhos

Total de despesas

In [46]:
seleciona['VR_DESPESA_CONTRATADA'].sum()

174111.21

Soma por fornecedor

In [47]:
seleciona.groupby(["NM_FORNECEDOR_RFB", "DS_ORIGEM_DESPESA"]).VR_DESPESA_CONTRATADA.sum().reset_index().sort_values("VR_DESPESA_CONTRATADA", ascending=False)

,NM_FORNECEDOR_RFB,DS_ORIGEM_DESPESA,VR_DESPESA_CONTRATADA
3,NASCIMENTO FELIX PROPAGANDA E PUBLICIDADE LTDA,Criação e inclusão de páginas na internet,"150,000.0000"
4,ROSIANA DE OLIVEIRA LEITE,Serviços prestados por terceiros,"15,369.3600"
1,#NULO#,Encargos sociais,"4,906.1500"
2,CARINE SETTI TOFFANO BARBOSA,Despesas com pessoal,"1,900.0000"
5,SEBASTIAO MARCOS CARVALHO FRANCISCO,Despesas com pessoal,"1,900.0000"
0,#NULO#,"Encargos financeiros, taxas bancárias e/ou op....",35.7000


## Faz o mesmo com deputados estaduais

Custo dos eleitos

In [56]:
candidatos_dep_est_eleitos = candidatos[(candidatos['DS_CARGO'] == 'DEPUTADO ESTADUAL') & 
                                        (candidatos['DS_SITUACAO_CANDIDATURA'] == 'APTO') &
                                        ((candidatos['DS_SIT_TOT_TURNO'] == 'ELEITO POR MÉDIA') | \
                                         (candidatos['DS_SIT_TOT_TURNO'] == 'ELEITO POR QP'))]

votos_eleitos_dep_est = pd.merge(candidatos_dep_est_eleitos, \
                                 conta, \
                                 left_on='SQ_CANDIDATO', \
                                 right_on='SQ_CANDIDATO')

votos_eleitos_dep_est_doacoes_fundos = pd.merge(votos_eleitos_dep_est, \
                                                soma, \
                                                left_on='SQ_CANDIDATO', \
                                                right_on='SQ_CANDIDATO')

def custo(linha):
    votos = linha["QT_VOTOS_NOMINAIS"]
    valor = linha["VR_RECEITA_FUNDOS"]
    if votos != 0 and valor !=0:
        return valor / votos 
    else:
        return valor

votos_eleitos_dep_est_doacoes_fundos["custo_por_voto"] = votos_eleitos_dep_est_doacoes_fundos.apply(custo, axis=1)

votos_eleitos_dep_est_doacoes_fundos.reset_index().sort_values("custo_por_voto", ascending=False).head(10)

,index,SG_UF,DS_CARGO,SQ_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,NR_CPF_CANDIDATO,DS_SITUACAO_CANDIDATURA,DS_DETALHE_SITUACAO_CAND,SG_PARTIDO,DT_NASCIMENTO,NR_IDADE_DATA_POSSE,NR_TITULO_ELEITORAL_CANDIDATO,DS_GENERO,DS_SIT_TOT_TURNO,QT_VOTOS_NOMINAIS,VR_RECEITA_FUNDOS,custo_por_voto
222,222,AP,DEPUTADO ESTADUAL,30000602639,LUCIANA ARAUJO GÓES GURGEL,LUCIANA,74066137253,APTO,DEFERIDO,PR,22/10/1982,36,044806761309,FEMININO,ELEITO POR QP,5825,"940,000.0000",161.3734
572,572,RR,DEPUTADO ESTADUAL,230000605257,LENIR RODRIGUES SANTOS,LENIR RODRIGUES,15394174253,APTO,DEFERIDO,PPS,23/04/1963,55,000239862607,FEMININO,ELEITO POR QP,4947,"749,548.0400",151.5157
492,492,AP,DEPUTADO ESTADUAL,30000620081,MARIA CRISTINA DO ROSARIO ALMEIDA MENDES,CRISTINA ALMEIDA,25141619249,APTO,DEFERIDO,PSB,27/05/1966,52,000139522518,FEMININO,ELEITO POR QP,3052,"400,000.0000",131.0616
192,192,AP,DEPUTADO ESTADUAL,30000602630,CHARLY JHONE SANTOS DE SOUSA,CHARLY JHONE,20915730278,APTO,DEFERIDO,PR,22/07/1969,49,003094862534,MASCULINO,ELEITO POR MÉDIA,2595,"335,000.0000",129.0944
545,545,TO,DEPUTADO ESTADUAL,270000626099,CLAUDIA TELLES DE MENEZES PIRES MARTINS LELIS,CLAUDIA LELIS,58423184153,APTO,DEFERIDO,PV,27/02/1972,46,008428472062,FEMININO,ELEITO POR MÉDIA,6746,"840,860.4700",124.6458
484,484,AP,DEPUTADO ESTADUAL,30000602634,JOSE CARLOS CARVALHO BARBOSA,KAKÁ BARBOSA,43772838472,APTO,DEFERIDO,PR,13/01/1966,52,000134022534,MASCULINO,ELEITO POR QP,7201,"850,000.0000",118.0392
552,552,RR,DEPUTADO ESTADUAL,230000602452,CATARINA DE LIMA GUERRA DA SILVA,CATARINA GUERRA,83986499253,APTO,DEFERIDO,SOLIDARIEDADE,28/01/1986,32,003314542658,FEMININO,ELEITO POR MÉDIA,4897,"555,000.0000",113.3347
517,517,RR,DEPUTADO ESTADUAL,230000625991,FRANCISCA AURELINA DE MEDEIROS LIMA,AURELINA MEDEIROS,06152619300,APTO,DEFERIDO,PODE,21/05/1951,67,000188852690,FEMININO,ELEITO POR MÉDIA,4864,"500,000.0000",102.7961
267,267,AC,DEPUTADO ESTADUAL,10000600792,JULIANA RODRIGUES DE OLIVEIRA,DOUTORA JULIANA,07882440272,APTO,DEFERIDO,PRB,30/03/1959,59,000689132402,FEMININO,ELEITO POR MÉDIA,5990,"587,600.0000",98.0968
260,260,AP,DEPUTADO ESTADUAL,30000625384,TELMA LUCIA DE AZEVEDO GURGEL,TELMA GURGEL,18222927272,APTO,DEFERIDO,PRP,02/05/1955,63,003511302585,FEMININO,ELEITO POR QP,5601,"495,000.0000",88.3771


Maior custo então foi de R$ 161,37

Verifica de todos os candidatos a deputado estadual

In [57]:
candidatos_dep_est = candidatos[(candidatos['DS_CARGO'] == 'DEPUTADO ESTADUAL') & 
                                        (candidatos['DS_SITUACAO_CANDIDATURA'] == 'APTO')]

votos_todos_dep_est = pd.merge(candidatos_dep_est, \
                               conta, \
                               left_on='SQ_CANDIDATO', \
                               right_on='SQ_CANDIDATO')

votos_todos_doacoes_fundos_dep_est = pd.merge(votos_todos_dep_est, \
                                              soma, \
                                              left_on='SQ_CANDIDATO', \
                                              right_on='SQ_CANDIDATO')

def custo(linha):
    votos = linha["QT_VOTOS_NOMINAIS"]
    valor = linha["VR_RECEITA_FUNDOS"]
    if votos != 0 and valor !=0:
        return valor / votos 
    else:
        return valor

votos_todos_doacoes_fundos_dep_est["custo_por_voto"] = votos_todos_doacoes_fundos_dep_est.apply(custo, axis=1)

votos_todos_doacoes_fundos_dep_est.reset_index().sort_values("custo_por_voto", ascending=False).head(10)

,index,SG_UF,DS_CARGO,SQ_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,NR_CPF_CANDIDATO,DS_SITUACAO_CANDIDATURA,DS_DETALHE_SITUACAO_CAND,SG_PARTIDO,DT_NASCIMENTO,NR_IDADE_DATA_POSSE,NR_TITULO_ELEITORAL_CANDIDATO,DS_GENERO,DS_SIT_TOT_TURNO,QT_VOTOS_NOMINAIS,VR_RECEITA_FUNDOS,custo_por_voto
64,64,SP,DEPUTADO ESTADUAL,250000619460,ANTONIO DIRCEU DALBEN,DALBEN,02507590807,APTO,INDEFERIDO COM RECURSO,PR,13/06/1964,54,109599230132,MASCULINO,NÃO ELEITO,0,"242,156.0300","242,156.0300"
936,936,MA,DEPUTADO ESTADUAL,100000605357,ZILMARA DE JESUS LIMA,ZILMARA LIMA,02241727350,APTO,DEFERIDO,PTB,09/08/1986,32,043356121155,FEMININO,SUPLENTE,119,"20,000,002.8900","168,067.2512"
5743,5743,MA,DEPUTADO ESTADUAL,100000601212,ROSANGELA APARECIDA BARROS CURADO,ROSANGELA CURADO,23671521287,APTO,DEFERIDO,PATRI,11/12/1966,52,026706281180,FEMININO,SUPLENTE,0,"150,000.0000","150,000.0000"
8812,8812,RO,DEPUTADO ESTADUAL,220000622093,CASSIANE ANDRADE ALVES,CASSIANE ANDRADE,80003303268,APTO,DEFERIDO COM RECURSO,DEM,15/06/1985,33,012564992305,FEMININO,NÃO ELEITO,0,"143,500.0000","143,500.0000"
5972,5972,MS,DEPUTADO ESTADUAL,120000615427,NATÁLYA RAMOS QUINTANA,NATÁLYA QUINTANA,05163989111,APTO,INDEFERIDO COM RECURSO,PP,20/05/1994,24,024029031937,FEMININO,NÃO ELEITO,0,"129,620.0000","129,620.0000"
3831,3831,RJ,DEPUTADO ESTADUAL,190000601368,JOSÉ CAMILO ZITO DOS SANTOS FILHO,ZITO,44154828720,APTO,INDEFERIDO COM RECURSO,PP,14/12/1952,66,049580410388,MASCULINO,NÃO ELEITO,0,"114,561.5400","114,561.5400"
7700,7700,MA,DEPUTADO ESTADUAL,100000601761,HEMETERIO WEBA FILHO,HEMETERIO WEBA,02939088349,APTO,INDEFERIDO COM RECURSO,PP,03/03/1951,67,009874961180,MASCULINO,NÃO ELEITO,0,"100,000.0000","100,000.0000"
6599,6599,PB,DEPUTADO ESTADUAL,150000617641,CARLOS ALBERTO BATINGA CHAVES,CARLOS BATINGA,04872010434,APTO,INDEFERIDO COM RECURSO,PTB,31/05/1951,67,006076921201,MASCULINO,NÃO ELEITO,0,"80,000.0000","80,000.0000"
7809,7809,SP,DEPUTADO ESTADUAL,250000619522,WILSON MASAYUKI ICHIOKA,KAZU REIS,07554837826,APTO,INDEFERIDO COM RECURSO,PR,04/01/1966,53,038034120132,MASCULINO,NÃO ELEITO,0,"53,511.7500","53,511.7500"
206,206,RS,DEPUTADO ESTADUAL,210000604709,GILMAR SOSSELLA,GILMAR SOSSELLA,33596336015,APTO,INDEFERIDO COM RECURSO,PDT,13/04/1961,57,046332500426,MASCULINO,NÃO ELEITO,0,"50,000.0000","50,000.0000"


Seleciona os não eleitos e custo por voto maior ou igual a R$ 1.613

In [58]:
nao_eleitos = votos_todos_doacoes_fundos_dep_est[(votos_todos_doacoes_fundos_dep_est['DS_SIT_TOT_TURNO'] == 'NÃO ELEITO') | 
                                         (votos_todos_doacoes_fundos_dep_est['DS_SIT_TOT_TURNO'] == 'SUPLENTE')].copy()

seleciona = nao_eleitos['custo_por_voto'] >= 1613

valores_altos = nao_eleitos[seleciona]

In [59]:
valores_altos.reset_index().sort_values("custo_por_voto", ascending=False).head(10)

,index,SG_UF,DS_CARGO,SQ_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,NR_CPF_CANDIDATO,DS_SITUACAO_CANDIDATURA,DS_DETALHE_SITUACAO_CAND,SG_PARTIDO,DT_NASCIMENTO,NR_IDADE_DATA_POSSE,NR_TITULO_ELEITORAL_CANDIDATO,DS_GENERO,DS_SIT_TOT_TURNO,QT_VOTOS_NOMINAIS,VR_RECEITA_FUNDOS,custo_por_voto
0,64,SP,DEPUTADO ESTADUAL,250000619460,ANTONIO DIRCEU DALBEN,DALBEN,02507590807,APTO,INDEFERIDO COM RECURSO,PR,13/06/1964,54,109599230132,MASCULINO,NÃO ELEITO,0,"242,156.0300","242,156.0300"
9,936,MA,DEPUTADO ESTADUAL,100000605357,ZILMARA DE JESUS LIMA,ZILMARA LIMA,02241727350,APTO,DEFERIDO,PTB,09/08/1986,32,043356121155,FEMININO,SUPLENTE,119,"20,000,002.8900","168,067.2512"
51,5743,MA,DEPUTADO ESTADUAL,100000601212,ROSANGELA APARECIDA BARROS CURADO,ROSANGELA CURADO,23671521287,APTO,DEFERIDO,PATRI,11/12/1966,52,026706281180,FEMININO,SUPLENTE,0,"150,000.0000","150,000.0000"
94,8812,RO,DEPUTADO ESTADUAL,220000622093,CASSIANE ANDRADE ALVES,CASSIANE ANDRADE,80003303268,APTO,DEFERIDO COM RECURSO,DEM,15/06/1985,33,012564992305,FEMININO,NÃO ELEITO,0,"143,500.0000","143,500.0000"
53,5972,MS,DEPUTADO ESTADUAL,120000615427,NATÁLYA RAMOS QUINTANA,NATÁLYA QUINTANA,05163989111,APTO,INDEFERIDO COM RECURSO,PP,20/05/1994,24,024029031937,FEMININO,NÃO ELEITO,0,"129,620.0000","129,620.0000"
35,3831,RJ,DEPUTADO ESTADUAL,190000601368,JOSÉ CAMILO ZITO DOS SANTOS FILHO,ZITO,44154828720,APTO,INDEFERIDO COM RECURSO,PP,14/12/1952,66,049580410388,MASCULINO,NÃO ELEITO,0,"114,561.5400","114,561.5400"
75,7700,MA,DEPUTADO ESTADUAL,100000601761,HEMETERIO WEBA FILHO,HEMETERIO WEBA,02939088349,APTO,INDEFERIDO COM RECURSO,PP,03/03/1951,67,009874961180,MASCULINO,NÃO ELEITO,0,"100,000.0000","100,000.0000"
64,6599,PB,DEPUTADO ESTADUAL,150000617641,CARLOS ALBERTO BATINGA CHAVES,CARLOS BATINGA,04872010434,APTO,INDEFERIDO COM RECURSO,PTB,31/05/1951,67,006076921201,MASCULINO,NÃO ELEITO,0,"80,000.0000","80,000.0000"
77,7809,SP,DEPUTADO ESTADUAL,250000619522,WILSON MASAYUKI ICHIOKA,KAZU REIS,07554837826,APTO,INDEFERIDO COM RECURSO,PR,04/01/1966,53,038034120132,MASCULINO,NÃO ELEITO,0,"53,511.7500","53,511.7500"
2,206,RS,DEPUTADO ESTADUAL,210000604709,GILMAR SOSSELLA,GILMAR SOSSELLA,33596336015,APTO,INDEFERIDO COM RECURSO,PDT,13/04/1961,57,046332500426,MASCULINO,NÃO ELEITO,0,"50,000.0000","50,000.0000"


In [60]:
valores_altos.groupby('DS_DETALHE_SITUACAO_CAND').size()

DS_DETALHE_SITUACAO_CAND
DEFERIDO                  54
DEFERIDO COM RECURSO       1
INDEFERIDO COM RECURSO    43
dtype: int64

In [61]:
valores_altos.groupby('QT_VOTOS_NOMINAIS').size()

QT_VOTOS_NOMINAIS
0      50
1       9
2       3
3       2
5       1
6       3
7       1
9       3
12      1
31      2
37      1
41      2
42      1
47      1
49      1
51      1
55      1
64      1
70      1
73      1
75      1
88      1
90      1
99      1
103     1
104     1
119     1
161     1
186     1
200     1
208     1
531     1
dtype: int64

Seleciona apenas quem não teve zero votos

In [62]:
valores_altos = valores_altos[(valores_altos['QT_VOTOS_NOMINAIS'] != 0)]

In [63]:
valores_altos.reset_index().sort_values("custo_por_voto", ascending=False).head(10)

,index,SG_UF,DS_CARGO,SQ_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,NR_CPF_CANDIDATO,DS_SITUACAO_CANDIDATURA,DS_DETALHE_SITUACAO_CAND,SG_PARTIDO,DT_NASCIMENTO,NR_IDADE_DATA_POSSE,NR_TITULO_ELEITORAL_CANDIDATO,DS_GENERO,DS_SIT_TOT_TURNO,QT_VOTOS_NOMINAIS,VR_RECEITA_FUNDOS,custo_por_voto
2,936,MA,DEPUTADO ESTADUAL,100000605357,ZILMARA DE JESUS LIMA,ZILMARA LIMA,02241727350,APTO,DEFERIDO,PTB,09/08/1986,32,043356121155,FEMININO,SUPLENTE,119,"20,000,002.8900","168,067.2512"
10,2455,AC,DEPUTADO ESTADUAL,10000629192,SONIA DE FATIMA SILVA ALVES,SONIA,41186257253,APTO,DEFERIDO,DEM,17/12/1974,44,002450462410,FEMININO,SUPLENTE,6,"240,300.0000","40,050.0000"
16,3504,AP,DEPUTADO ESTADUAL,30000623195,HELEN MACHADO ARAUJO,HELEM BELLE,00826223290,APTO,DEFERIDO,PTC,02/11/1992,26,006052322500,FEMININO,SUPLENTE,1,"20,000.0000","20,000.0000"
13,2881,RR,DEPUTADO ESTADUAL,230000608357,MILENE ALVES DO NASCIMENTO,MILENE NASCIMENTO,03518652257,APTO,DEFERIDO,PR,08/03/1997,21,004646372615,FEMININO,SUPLENTE,3,"40,000.0000","13,333.3333"
25,5719,AC,DEPUTADO ESTADUAL,10000600153,VANUSA FERREIRA VERAS,VANUSA VERAS,78150507272,APTO,DEFERIDO,PRP,20/10/1983,35,004431842470,FEMININO,SUPLENTE,1,"10,073.3400","10,073.3400"
45,8811,AC,DEPUTADO ESTADUAL,10000600804,VANUSA FERREIRA CORREA,VANUSA CORREA,51215276249,APTO,DEFERIDO,PRB,21/01/1977,42,003027292453,FEMININO,SUPLENTE,1,"10,000.0000","10,000.0000"
42,8319,AP,DEPUTADO ESTADUAL,30000623188,ALBA CILENE SOUZA,CILENE SOUZA,85920908220,APTO,DEFERIDO,PTC,23/06/1984,34,003784812593,FEMININO,SUPLENTE,2,"20,000.0000","10,000.0000"
29,6269,PI,DEPUTADO ESTADUAL,180000628749,TAMIRES ALMEIDA VASCONCELOS,TAMIRES VASCONCELOS,06336579301,APTO,DEFERIDO,PR,02/11/1996,22,042746231503,FEMININO,SUPLENTE,41,"370,700.0000","9,041.4634"
47,9080,RR,DEPUTADO ESTADUAL,230000629632,REGIANE DA FONSECA BRAGA,REGIANE BRAGA,97938467200,APTO,DEFERIDO,PSD,10/03/1990,28,058610431309,FEMININO,NÃO ELEITO,9,"80,000.0000","8,888.8889"
46,8849,RR,DEPUTADO ESTADUAL,230000609358,ALICE SILVA,ALICE SILVA,02807429246,APTO,DEFERIDO,PV,05/06/1997,21,004551412690,FEMININO,SUPLENTE,1,"7,703.1200","7,703.1200"


Aqui consta que a candidata ZILMARA DE JESUS LIMA recebeu R$ 20 milhões do Fundo Especial <br>
É provável que seja erro de digitação na prestação de contas, mas o TSE pode confirmar

Checa alguns gastos

ZILMARA DE JESUS LIMA

In [64]:
seleciona = despesas[(despesas['SQ_CANDIDATO'] == '100000605357')]

seleciona['VR_DESPESA_CONTRATADA'].sum()

19000.0

In [65]:
seleciona.groupby(["NM_FORNECEDOR_RFB", "DS_ORIGEM_DESPESA"]).VR_DESPESA_CONTRATADA.sum().reset_index().sort_values("VR_DESPESA_CONTRATADA", ascending=False)

,NM_FORNECEDOR_RFB,DS_ORIGEM_DESPESA,VR_DESPESA_CONTRATADA
0,PARTIDO TRABALHISTA BRASILEIRO,Publicidade por materiais impressos,"19,000.0000"


In [66]:
seleciona.groupby(["NM_FORNECEDOR_RFB", "DS_ORIGEM_DESPESA", "DS_DESPESA"]).VR_DESPESA_CONTRATADA.sum().reset_index().sort_values("VR_DESPESA_CONTRATADA", ascending=False)

,NM_FORNECEDOR_RFB,DS_ORIGEM_DESPESA,DS_DESPESA,VR_DESPESA_CONTRATADA
9,PARTIDO TRABALHISTA BRASILEIRO,Publicidade por materiais impressos,CONFECÇÃO DE SANTINHOS 4X0 6X9 CM,"6,000.0000"
0,PARTIDO TRABALHISTA BRASILEIRO,Publicidade por materiais impressos,ADESIVO 40X50 CM AUTOMOTIVO,"2,400.0000"
2,PARTIDO TRABALHISTA BRASILEIRO,Publicidade por materiais impressos,ADESIVO PERFURADO 120 CM X 60 CM,"2,000.0000"
3,PARTIDO TRABALHISTA BRASILEIRO,Publicidade por materiais impressos,ADESIVO PERFURADO 1·20 X 0·60 CM,"1,800.0000"
6,PARTIDO TRABALHISTA BRASILEIRO,Publicidade por materiais impressos,CARTAZ 33X45 CM POLICROMIA,"1,320.0000"
5,PARTIDO TRABALHISTA BRASILEIRO,Publicidade por materiais impressos,BANDEIRA EM CETIN MED 60X90,"1,280.0000"
8,PARTIDO TRABALHISTA BRASILEIRO,Publicidade por materiais impressos,CONFECÇÃO DE CARTAZ 4X0 TAMANHO 45X33 CM,"1,200.0000"
1,PARTIDO TRABALHISTA BRASILEIRO,Publicidade por materiais impressos,ADESIVO 4X0 40X50 CM,"1,000.0000"
4,PARTIDO TRABALHISTA BRASILEIRO,Publicidade por materiais impressos,ADESIVO REDONDO 6X6 (BOTOM),"1,000.0000"
7,PARTIDO TRABALHISTA BRASILEIRO,Publicidade por materiais impressos,CARTAZ 42X62 CM POLICROMIA,"1,000.0000"


SONIA DE FATIMA SILVA ALVES

In [67]:
seleciona = despesas[(despesas['SQ_CANDIDATO'] == '10000629192')]

seleciona['VR_DESPESA_CONTRATADA'].sum()

240159.22999999998

In [68]:
seleciona.groupby(["NM_FORNECEDOR_RFB", "DS_ORIGEM_DESPESA"]).VR_DESPESA_CONTRATADA.sum().reset_index().sort_values("VR_DESPESA_CONTRATADA", ascending=False)

,NM_FORNECEDOR_RFB,DS_ORIGEM_DESPESA,VR_DESPESA_CONTRATADA
35,JOSE ULINEIDE BENIGNO GOMES,Atividades de militância e mobilização de rua,"10,000.0000"
26,JACOB GOMES DE ALMEIDA JUNIOR 65243935204,Publicidade por materiais impressos,"10,000.0000"
12,DAUSTER MACIEL NETO,Atividades de militância e mobilização de rua,"10,000.0000"
41,MARCOS ANDRE BAIMA CATAO,Atividades de militância e mobilização de rua,"7,000.0000"
56,NESIO MENDES DE CARVALHO,Cessão ou locação de veículos,"6,000.0000"
21,GENIVALDO VIEIRA,Atividades de militância e mobilização de rua,"6,000.0000"
67,WEVERSON GONDIM DA SILVA,Atividades de militância e mobilização de rua,"5,000.0000"
16,ERICA DE MOURA SILVA,Atividades de militância e mobilização de rua,"5,000.0000"
44,MARIA ANTONIA DA SILVA ARAUJO,Atividades de militância e mobilização de rua,"5,000.0000"
42,MARCOS MOURA DOS SANTOS,Atividades de militância e mobilização de rua,"5,000.0000"


HELEN MACHADO ARAUJO

In [69]:
seleciona = despesas[(despesas['SQ_CANDIDATO'] == '30000623195')]

seleciona['VR_DESPESA_CONTRATADA'].sum()

19750.0

In [70]:
seleciona.groupby(["NM_FORNECEDOR_RFB", "DS_ORIGEM_DESPESA"]).VR_DESPESA_CONTRATADA.sum().reset_index().sort_values("VR_DESPESA_CONTRATADA", ascending=False)

,NM_FORNECEDOR_RFB,DS_ORIGEM_DESPESA,VR_DESPESA_CONTRATADA
1,M R COM E SERVICOS LTDA,Publicidade por materiais impressos,"14,800.0000"
0,FABIO WILSON AMANAJAS PAES,Serviços prestados por terceiros,"4,950.0000"
